In [ ]:
# 코랩용 설정
!git clone https://github.com/Team-M1/preprocessing-for-final-project.git
%cd preprocessing-for-final-project
!pip install -r requirements.txt

In [ ]:
# 코랩용 설정

from google.colab import drive
drive.mount('/content/drive')

save_path = "/content/drive/MyDrive/Colab Notebooks/model"

In [1]:
import os
import numpy as np
import pandas as pd
import torch
from torch import nn


device = "cuda:0" if torch.cuda.is_available() else "cpu"

In [1]:
# 데이터 불러오기

df = pd.read_csv("./data/혐오표현_정제.csv")
df.head(2)

,content,hate,gender_hate
0,걍 이 병신나라 민도가 좆박은건데 어쩔수가 잇나 걍 망해야지,1,0
1,그 앰생 쓰레기 병신들은 짐승이고. 니는 짐승을 사람 취급해주냐.,1,0


In [3]:
# 만약 클래스별 가중치를 구하겠다면 사용

# weight = df.shape[0] / (len(df["gender_hate"].unique()) * np.bincount(df["gender_hate"]))
# weight = torch.from_numpy(weight).float()
# weight

tensor([0.5651, 4.3406])

In [3]:
# 모델 불러오기

from transformers import ElectraForSequenceClassification, ElectraTokenizer
from tokenization_kocharelectra import KoCharElectraTokenizer


electramodel = ElectraForSequenceClassification.from_pretrained("monologg/kocharelectra-small-discriminator")
tokenizer = KoCharElectraTokenizer.from_pretrained("monologg/kocharelectra-small-discriminator")
electramodel = electramodel.to(device)

Some weights of the model checkpoint at monologg/kocharelectra-small-discriminator were not used when initializing ElectraForSequenceClassification: ['discriminator_predictions.dense.bias', 'discriminator_predictions.dense.weight', 'discriminator_predictions.dense_prediction.weight', 'discriminator_predictions.dense_prediction.bias']
- This IS expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing ElectraForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of ElectraForSequenceClassification were not initialized from the model checkpoint at monologg/kocharelectra-small-discriminator and are newly initializ

In [4]:
# 데이터프레임을 TensorDataset으로 만들기

from torch.utils.data import TensorDataset

from data_preprocess import df_to_feature_and_label


all_data = TensorDataset(*df_to_feature_and_label(df, tokenizer, max_length=256))

lr = 0.001

# criterion = torch.nn.CrossEntropyLoss(weight=weight.to(device))  # 클래스별 가중치가 적용된 크로스엔트로피로스
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(electramodel.classifier.parameters(), lr=lr)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

In [ ]:
# 코랩용 체크포인트 불러오기 설정
saved_checkpoint = os.path.join(save_path, "checkpoint210814.pth")
checkpoint = torch.load(saved_checkpoint)
electramodel.load_state_dict(checkpoint["model_state_dict"])
optimizer.load_state_dict(checkpoint["optimizer_state_dict"])
scheduler.load_state_dict(checkpoint["scheduler_state_dict"])

In [5]:
# ImbalancedDatasetSampler에서 사용되는 함수
# dataset[:][3]은 label값을 가리킨다.

def get_labels(dataset):
    return dataset[:][3]

In [ ]:
from trainer import training


training(
    all_data,
    electramodel,
    criterion,
    optimizer,
    scheduler,
    get_label=get_labels,
    epochs=100,
    batch_size=16,
    save_path=save_path
)